In [1]:
!pip install torch-scatter==latest+cu101 -f https://pytorch-geometric.com/whl/torch-1.4.0.html
!pip install torch-sparse==latest+cu101 -f https://pytorch-geometric.com/whl/torch-1.4.0.html
!pip install torch-cluster==latest+cu101-f https://pytorch-geometric.com/whl/torch-1.4.0.html
!pip install torch-spline-conv==latest+cu101 -f https://pytorch-geometric.com/whl/torch-1.4.0.html
!pip install torch-geometric==1.5.0


Looking in links: https://pytorch-geometric.com/whl/torch-1.4.0.html
  Attempting uninstall: torch-scatter
    Found existing installation: torch-scatter 1.1.1
    Uninstalling torch-scatter-1.1.1:
      Successfully uninstalled torch-scatter-1.1.1
Looking in links: https://pytorch-geometric.com/whl/torch-1.4.0.html
  Attempting uninstall: torch-sparse
    Found existing installation: torch-sparse 0.2.3
    Uninstalling torch-sparse-0.2.3:
      Successfully uninstalled torch-sparse-0.2.3
  Using cached https://pytorch-geometric.com/whl/torch-1.4.0.html (45 kB)
  ERROR: Cannot unpack file C:\Users\Alfredo\AppData\Local\Temp\pip-unpack-hzuv__49\torch-1.4.0.html (downloaded from C:\Users\Alfredo\AppData\Local\Temp\pip-req-build-uu2vymck, content-type: text/html); cannot detect archive format
ERROR: Cannot determine archive format of C:\Users\Alfredo\AppData\Local\Temp\pip-req-build-uu2vymck
Looking in links: https://pytorch-geometric.com/whl/torch-1.4.0.html
  Attempting uninstall: torch

In [4]:
import sys  
sys.path.insert(0, '/path/to/application/app/folder')

import torch
from torch_geometric.utils import one_hot
from torch_scatter import scatter_add
from torch_geometric.data import InMemoryDataset, Data, download_url, extract_zip

RuntimeError: Expected PyTorch version 1.4 but found version 1.0.

In [ ]:
# Dynamic Data for torch geometric
class MyDynamicDataset(Dataset):
    def __init__(
        self,
        root,
        A,
        links,
        labels,
        h,
        sample_ratio,
        max_nodes_per_hop,
        u_features,
        v_features,
        class_values,
        max_num=None,
    ):
        super(MyDynamicDataset, self).__init__(root)
        self.A = A
        self.links = links
        self.labels = labels
        self.h = h
        self.sample_ratio = sample_ratio
        self.max_nodes_per_hop = max_nodes_per_hop
        self.u_features = u_features
        self.v_features = v_features
        self.class_values = class_values
        if max_num is not None:
            np.random.seed(123)
            num_links = len(links[0])
            perm = np.random.permutation(num_links)
            perm = perm[:max_num]
            self.links = (links[0][perm], links[1][perm])
            self.labels = labels[perm]

    def __len__(self):
        return len(self.links[0])

    def get(self, idx):
        # node_features = [self.u_features[0], self.v_features[0]]
        nodes_distances_tpl = []
        i, j = self.links[0][idx], self.links[1][idx]
        g_label = self.labels[idx]

        # i and j indices
        nodes, distances = subgraph_extraction(
            (i, j), self.A, self.h, self.sample_ratio, self.max_nodes_per_hop
        )
        nodes_distances_tpl.append((nodes, distances))

        # side matrix index (item features)
        k = random_nonzero(j, self.v_features)  # TODO: could this be an issue?
        nodes, distances = subgraph_extraction(
            (j, k),
            self.v_features,
            self.h,
            self.sample_ratio,
            self.max_nodes_per_hop,
            g_label,
        )
        nodes_distances_tpl.append((nodes, distances))
        nodes = [nd[0] for nd in nodes_distances_tpl]
        distances = [nd[1] for nd in nodes_distances_tpl]
        matrices = [self.A, self.v_features]  # main, side

        # node labeling
        tmp = subgraph_labeling(
            nodes,
            distances,
            matrices,
            class_values=self.class_values,
            h=self.h,
            g_label=g_label,
        )

        return construct_pyg_graph(*tmp)

In [ ]:
# Function to select an nonzero index randomly from matrix.
def random_nonzero(index, matrix):
    tpl = np.nonzero(matrix[index])
    return random.choice(tpl[1])  # because the first index will always be 0

In [ ]:

"""
    Method to extract the subgraph from collective matrices
"""
def subgraph_extraction(
    inds, A, h=1, sample_ratio=1.0, max_nodes_per_hop=None, class_values=None
):
    # extract the h-hop enclosing subgraph around link 'ind'
    dist = 0
    u_index = inds[0]
    i_index = inds[1]

    nodes = [[u_index], [i_index]]
    distances = [[0], [0]]  # O(M)
    visited_sets = [set([u_index]), set([i_index])]  # O(M)
    fringe_sets = [set([u_index]), set([i_index])]  # O(M)
    for dist in range(1, h + 1):  # for now, we only focus in 1-hop
        # get fringe with neighbors(node,A,is_row)
        fringe_sets[1], fringe_sets[0] = neighbors(fringe_sets[0], A, True), neighbors(
            fringe_sets[1], A, False
        )

        # update fringe based on visited sets
        fringe_sets[0] = fringe_sets[0] - visited_sets[0]
        fringe_sets[1] = fringe_sets[1] - visited_sets[1]

        # get visited nodes based on u and v fringes
        visited_sets[0] = visited_sets[0].union(fringe_sets[0])
        visited_sets[1] = visited_sets[1].union(fringe_sets[1])

        # use sample ratio (if defined)
        if sample_ratio < 1.0:
            fringe_sets[0] = random.sample(
                fringe_sets[0], int(sample_ratio * len(fringe_sets[0]))
            )
            fringe_sets[1] = random.sample(
                fringe_sets[1], int(sample_ratio * len(fringe_sets[1]))
            )

        # limiting the number of nodes_per hop (if defined)
        if max_nodes_per_hop is not None:
            if max_nodes_per_hop < len(fringe_sets[0]):
                fringe_sets[0] = random.sample(fringe_sets[0], max_nodes_per_hop)
            if max_nodes_per_hop < len(fringe_sets[1]):
                fringe_sets[1] = random.sample(fringe_sets[1], max_nodes_per_hop)

        # stop if there are no fringes on u and v
        if len(fringe_sets[0]) == 0 and len(fringe_sets[1]) == 0:
            break

        # update u and v nodes
        nodes[0] = nodes[0] + list(fringe_sets[0])
        nodes[1] = nodes[1] + list(fringe_sets[1])

        # update u and v distances
        distances[0] = distances[0] + [dist] * len(fringe_sets[0])
        distances[1] = distances[1] + [dist] * len(fringe_sets[1])

    return nodes, distances


In [ ]:

def subgraph_labeling(raw_nodes, raw_distances, matrices, class_values, h=1, g_label=1):
    # TODO: make it dynamic by handling multiple matrices later
    y_genre = 0  # genre is always because it was selected to be 1 before.

    u_nodes, v_nodes, w_nodes = raw_nodes[0][0], raw_nodes[0][1], raw_nodes[1][1]
    u_dist, v_dist, w_dist = (
        raw_distances[0][0],
        raw_distances[0][1],
        raw_distances[1][1],
    )
    nodes = [u_nodes, v_nodes, w_nodes]
    distances = [u_dist, v_dist, w_dist]

    subgraphs = []
    for i in range(1, len(matrices) + 1):
        subgraph = matrices[i - 1][nodes[i - 1], :][:, nodes[i]]
        subgraph[0, 0] = 0
        subgraphs.append(subgraph)

    u, v, r = ssp.find(subgraphs[0])  # r is 1, 2... (rating labels + 1)
    z, w, sg = ssp.find(subgraphs[1])  # sg (side genre) is 1 (genre exist)
    r = r.astype(int)
    sg = sg.astype(int)

    v += len(u_nodes)  # starting point
    # shift z. 
    w += len(u_nodes) + len(v_nodes)  # starting point

    y1 = class_values[g_label]
    y2 = y_genre

    r = r - 1  # transform r back to rating label
    sg = sg  # transform rating side back to original label

    # Node-labeling process
    node_labels = []
    for i in range(len(distances)):
        node_labels += [h * len(distances) + i for h in distances[i]]

    # Set max node label (2 matrix, 3 relations, 1 hop)
    max_node_label = h * (len(matrices) * len(distances))
    indexes = {"u": u, "v": v, "z": z, "w": w}
    y = [y1, y2]
    ratings = [r, sg]

    return indexes, node_labels, max_node_label, y, ratings


In [ ]:

def construct_pyg_graph(indexes, node_labels, max_node_label, y, ratings):
    # (u,v,r), (v,w,rs)

    u, v, w, z = indexes["u"], indexes["v"], indexes["w"], indexes["z"]
    u, v, w, z = (
        torch.LongTensor(u),
        torch.LongTensor(v),
        torch.LongTensor(w),
        torch.LongTensor(z),
    )
    r, sg = torch.LongTensor(ratings[0]), torch.LongTensor(ratings[1])
    # I am not sure how to include this
    # z = item prime
    # w = genre
    # u = user
    # v = item
    edge_index = torch.stack([torch.cat([u, v]), torch.cat([v, u ])], 0)
    edge_type = torch.cat([r, r, sg])  # why it works?
    y1 = torch.FloatTensor([y[0]])
    y2 = torch.LongTensor([y[1]])
    x = torch.FloatTensor(one_hot(node_labels, max_node_label + 1))
    data = Data(x, edge_index, edge_type=edge_type, y1=y1, y2=y2)
    return data

# Execute

In [ ]:
from ../preprocessing.py import load_official_trainvaltest_split

rating_map, post_rating_map = None, {
        x: int(i // (5 / 5))
        for i, x in enumerate(np.arange(1, 6).tolist())
    }  # refer to IGMC to use rating map

isTesting = False

(u_features, v_features, adj_train, train_labels, train_u_indices train_v_indices,val_labels, val_u_indices, val_v_indices, test_labels,test_u_indices, test_v_indices, class_values) = load_official_trainvaltest_split("ml_100k", isTesting, rating_map, post_rating_map, 1.0)

# prepare indices
train_indices = (train_u_indices, train_v_indices)
val_indices = (val_u_indices, val_v_indices)
test_indices = (test_u_indices, test_v_indices)
print(
    "#train: %d, #val: %d, #test: %d"
    % (
        len(train_u_indices),
        len(val_u_indices),
        len(test_u_indices),
    )
)

In [ ]:
# Extract enclosing subgraph
train_graphs, val_graphs, test_graphs = None, None, None
data_combo = (args.data_name, args.data_appendix, val_test_appendix)

# create dataset, either dynamically extract enclosing subgraphs,
# or extract in preprocessing and save to disk.
dataset_class = "MyDynamicDataset" 
train_graphs = eval(dataset_class)(
    "data/{}{}/{}/train".format(*data_combo),
    adj_train,
    train_indices,
    train_labels,
    args.hop,
    args.sample_ratio,
    args.max_nodes_per_hop,
    u_features,
    v_features,
    class_values,
    max_num=args.max_train_num,
)
dataset_class = "MyDynamicDataset"
test_graphs = eval(dataset_class)(
    "data/{}{}/{}/test".format(*data_combo),
    adj_train,
    test_indices,
    test_labels,
    args.hop,
    args.sample_ratio,
    args.max_nodes_per_hop,
    u_features,
    v_features,
    class_values,
    max_num=args.max_test_num,
)
if not args.testing:
    dataset_class = "MyDynamicDataset" 
    val_graphs = eval(dataset_class)(
        "data/{}{}/{}/val".format(*data_combo),
        adj_train,
        val_indices,
        val_labels,
        args.hop,
        args.sample_ratio,
        args.max_nodes_per_hop,
        u_features,
        v_features,
        class_values,
        max_num=args.max_val_num,
    )

# Train

In [2]:
from models.py import IGCMF

num_relations = len(class_values)
multiply_by = 1
n_features = (0)  # NOTE: considering it is using CMF because the features become inputs

model = IGCMF( train_graphs,
    latent_dim=[32, 32, 32, 32],  # increase latent dimension to 128
    num_relations=num_relations,
    num_bases=4,
    regression=True,
    adj_dropout=args.adj_dropout,
    force_undirected=args.force_undirected,
    side_features=False,
    n_side_features=n_features,
    multiply_by=multiply_by)


ModuleNotFoundError: No module named &#39;models&#39;